# Linear Regression: A foray into machine Learning
by Michael Silverstein
msilvers@broadinstitute.org
github.com/michaelsilverstein/

While a relatively simple concept, linear regression, or fitting a line to a set of data, serves as a great conduit to a far more complex idea: machine learning. Here I outline how to generate a noisy-linear data set, learn features from that data set, and predict a line of best fit through that dataset in hopes of recovering the original linear relationship.
